In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
from pathlib import Path
sys.path.append("python_lib")

In [3]:
from tqdm.notebook import tqdm_notebook as tqdm
import nnsplit
from nnsplit import train, utils
train.tqdm = tqdm

In [4]:
cache_dir = Path("cache")
cache_dir.mkdir(exist_ok=True)

# Prepare data

In [7]:
train.prepare_data("train_data/dewiki-20180920-corpus.xml", "de", max_n_sentences=1_000_000, 
                   data_directory=cache / "de_data")

(tensor([[107, 106, 111,  ...,  86,  99, 105],
         [117,  34,  82,  ..., 102, 119, 116],
         [103, 112,  34,  ...,  71, 107, 112],
         ...,
         [ 99, 118, 107,  ..., 116, 111,  99],
         [106, 112, 107,  ..., 112, 113, 110],
         [105, 103, 117,  ..., 114, 106, 103]], dtype=torch.uint8),
 tensor([[[False, False],
          [False, False],
          [False, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [ True, False],
          [False, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         [[False, False],
          [False, False],
          [ True, False],
          ...,
          [False, False],
          [False, False],
          [False, False]],
 
         ...,
 
         [[False, False],
          [False, False],
          [False, False],
          ...,
          [False, False],
          [False, False],


# Train model

In [ ]:
learner = train.train_from_directory("cache/de_data", n_epochs=2)
learner

epoch,train_loss,valid_loss,time
0,0.003551,0.003577,01:30


In [7]:
utils.store_model(learner, "data/de")

# Test

In [5]:
from nnsplit import NNSplit

In [6]:
splitter = NNSplit(utils.load_model("cache/de_data"))

In [9]:
splitter.split(["Das ist ein Test Das ist noch ein Test."])

[[[Token(text='Das', whitespace=' '),
   Token(text='ist', whitespace=' '),
   Token(text='ein', whitespace=' '),
   Token(text='Test', whitespace=' ')],
  [Token(text='Das', whitespace=' '),
   Token(text='ist', whitespace=' '),
   Token(text='noch', whitespace=' '),
   Token(text='ein', whitespace=' '),
   Token(text='Test', whitespace='')],
  [Token(text='.', whitespace='')]]]